## Load Data

In [1]:
import pandas as pd

data = pd.read_csv("data.csv")
data = data["content"].drop_duplicates().dropna()[:37000]
print(data.info())
print(data.head())

<class 'pandas.core.series.Series'>
Int64Index: 37000 entries, 0 to 37044
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
37000 non-null  object
dtypes: object(1)
memory usage: 578.1+ KB
None
0    WASHINGTON  —   Congressional Republicans have...
1    After the bullet shells get counted, the blood...
2    When Walt Disney’s “Bambi” opened in 1942, cri...
3    Death may be the great equalizer, but it isn’t...
4    SEOUL, South Korea  —   North Korea’s leader, ...
Name: content, dtype: object


## Removing Stopwords

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('words')

stop_words = set(nltk.corpus.stopwords.words('english'))
corp = set(nltk.corpus.words.words())

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([word for word in textArr if word not in stop_words and word in corp])
    return rem_text

# remove stopwords from the text
data=data.apply(remove_stopwords)

print(data.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eslam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


0    new fear comes health care lawsuit might The i...
1    bullet get blood votive burn people peer see c...
2    Walt visual vastly different anything done kno...
3    may great necessarily endeavor mortal consider...
4    South Kim said Sunday country making final con...
Name: content, dtype: object


## Lemmatization

In [3]:
import spacy
from gensim import corpora

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output
tokenized_reviews = lemmatization(data.tolist())

print(tokenized_reviews[:2])

In [ ]:

# print number of tokenization
from importlib.util import find_spec as isModule
if(isModule('humanize') != None):
      from humanize import intword
      print(intword(sum(len(x) for x in tokenized_reviews)), " Tokenizations")
else:
      print(sum(len(x) for x in tokenized_reviews), " Tokenizations")

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

print(doc_term_matrix[:2])

## Model Training

In [ ]:
import gensim
# Creating the object for LDA model using gensim library
Lda = gensim.models.LdaMulticore

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=30, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))

---------- 
 Topics
 [(12, '0.029*"company" + 0.015*"business" + 0.012*"border" + 0.011*"new" + 0.009*"mexican" + 0.008*"executive" + 0.008*"financial" + 0.007*"chief" + 0.007*"food" + 0.007*"money"'), (19, '0.036*"immigration" + 0.024*"illegal" + 0.020*"order" + 0.018*"executive" + 0.015*"government" + 0.013*"country" + 0.013*"border" + 0.012*"administration" + 0.012*"ban" + 0.011*"new"'), (5, '0.024*"white" + 0.019*"people" + 0.017*"black" + 0.013*"man" + 0.010*"gay" + 0.008*"social" + 0.007*"actor" + 0.007*"time" + 0.006*"film" + 0.005*"woman"'), (24, '0.053*"police" + 0.034*"gun" + 0.026*"people" + 0.014*"law" + 0.012*"violence" + 0.010*"enforcement" + 0.010*"city" + 0.009*"fire" + 0.009*"man" + 0.008*"protest"'), (11, '0.042*"percent" + 0.015*"poll" + 0.013*"game" + 0.013*"last" + 0.013*"first" + 0.012*"team" + 0.010*"second" + 0.009*"good" + 0.009*"time" + 0.008*"final"'), (10, '0.017*"political" + 0.014*"man" + 0.011*"government" + 0.010*"many" + 0.009*"people" + 0.007*"power" +

## Evaluation

In [ ]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=len(data.index)))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Visualization

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis